In [17]:
!pip install plotly
!pip install scikit-learn
!pip install numpy
!pip install pandas

In [18]:
import sklearn as sk
import numpy as np
import plotly.express as px
import pandas as pd

In [45]:
# Loading Datasets
data = pd.read_csv('./Crime_Data_from_2020_to_Present.csv', usecols=['DR_NO','AREA NAME' ,'Date Rptd', 'DATE OCC', 'TIME OCC', 'LOCATION', 'Crm Cd Desc', 'Vict Age', 'Vict Sex', 'Weapon Desc', 'Premis Desc'], header=0)
data = data[data['DATE OCC'].str.contains('2024')]

encoded = pd.DataFrame()

data.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Desc,LOCATION
111485,240312995,08/08/2024 12:00:00 AM,08/07/2024 12:00:00 AM,2344,Southwest,VEHICLE - STOLEN,0,NaN,PARKING LOT,NaN,3000 S FIGUEROA ST
111596,240313195,08/14/2024 12:00:00 AM,08/13/2024 12:00:00 AM,1830,Southwest,VEHICLE - STOLEN,0,NaN,STREET,NaN,3500 CHESAPEAKE AV
111652,241810370,05/30/2024 12:00:00 AM,05/29/2024 12:00:00 AM,2000,Southeast,VEHICLE - STOLEN,0,NaN,STREET,NaN,2000 E 112TH ST
111760,241712034,08/14/2024 12:00:00 AM,08/13/2024 12:00:00 AM,2100,Devonshire,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),58,F,STREET,NaN,CHASE
449878,242104492,01/12/2024 12:00:00 AM,01/12/2024 12:00:00 AM,1830,Topanga,BURGLARY,35,M,SINGLE FAMILY DWELLING,NaN,23500 WINDOM ST


In [30]:
data.describe()

,DR_NO,TIME OCC,Vict Age
count,1.275720e+05,127572.000000,127572.000000
mean,2.410908e+08,1367.104278,23.471420
std,7.906350e+05,636.695441,22.440857
min,2.303006e+08,1.000000,0.000000
25%,2.405098e+08,930.000000,0.000000
50%,2.411060e+08,1445.000000,24.000000
75%,2.416049e+08,1900.000000,39.000000
max,2.521040e+08,2359.000000,99.000000


In [44]:
# Time Conversion

# Combine DATE OCC and TIME OCC into a single datetime column
data['DATE OCC'] = pd.to_datetime(data['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')
#data['DATE OCC'] = pd.to_datetime(data['DATE OCC'] + ' ' + data['TIME OCC'], format='%Y-%m-%d %M:%S')
# Convert the datetime column to Unix timestamps
data['UNIX_TIMESTAMP'] = data['DATE OCC'].apply(lambda x: x.timestamp())
data

,DR_NO,Date Rptd,DATE OCC,TIME OCC,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Desc,LOCATION,UNIX_TIMESTAMP
111485,240312995,08/08/2024 12:00:00 AM,2024-08-07,2344,VEHICLE - STOLEN,0,NaN,PARKING LOT,NaN,3000 S FIGUEROA ST,1.722989e+09
111596,240313195,08/14/2024 12:00:00 AM,2024-08-13,1830,VEHICLE - STOLEN,0,NaN,STREET,NaN,3500 CHESAPEAKE AV,1.723507e+09
111652,241810370,05/30/2024 12:00:00 AM,2024-05-29,2000,VEHICLE - STOLEN,0,NaN,STREET,NaN,2000 E 112TH ST,1.716941e+09
111760,241712034,08/14/2024 12:00:00 AM,2024-08-13,2100,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),58,F,STREET,NaN,CHASE,1.723507e+09
449878,242104492,01/12/2024 12:00:00 AM,2024-01-12,1830,BURGLARY,35,M,SINGLE FAMILY DWELLING,NaN,23500 WINDOM ST,1.705018e+09
...,...,...,...,...,...,...,...,...,...,...,...
1004891,240710284,07/24/2024 12:00:00 AM,2024-07-23,1400,VEHICLE - STOLEN,0,NaN,STREET,NaN,4000 W 23RD ST,1.721693e+09
1004892,240104953,01/15/2024 12:00:00 AM,2024-01-15,100,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0,X,HOTEL,UNKNOWN WEAPON/OTHER WEAPON,1300 W SUNSET BL,1.705277e+09
1004893,240410786,10/14/2024 12:00:00 AM,2024-10-11,2330,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",29,M,RESTAURANT/FAST FOOD,NaN,1700 ALBION ST,1.728605e+09
1004894,240309674,04/24/2024 12:00:00 AM,2024-04-24,1500,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",70,F,SIDEWALK,STICK,FLOWER ST,1.713917e+09


In [47]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the 'Crm Cd Desc' column
encoded['Crm Cd Desc'] = label_encoder.fit_transform(data['Crm Cd Desc'])
data

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA NAME,Crm Cd Desc,Vict Age,Vict Sex,Premis Desc,Weapon Desc,LOCATION,Crm Cd Desc Encoded
111485,240312995,08/08/2024 12:00:00 AM,08/07/2024 12:00:00 AM,2344,Southwest,VEHICLE - STOLEN,0,NaN,PARKING LOT,NaN,3000 S FIGUEROA ST,122
111596,240313195,08/14/2024 12:00:00 AM,08/13/2024 12:00:00 AM,1830,Southwest,VEHICLE - STOLEN,0,NaN,STREET,NaN,3500 CHESAPEAKE AV,122
111652,241810370,05/30/2024 12:00:00 AM,05/29/2024 12:00:00 AM,2000,Southeast,VEHICLE - STOLEN,0,NaN,STREET,NaN,2000 E 112TH ST,122
111760,241712034,08/14/2024 12:00:00 AM,08/13/2024 12:00:00 AM,2100,Devonshire,THEFT FROM MOTOR VEHICLE - PETTY ($950 & UNDER),58,F,STREET,NaN,CHASE,102
449878,242104492,01/12/2024 12:00:00 AM,01/12/2024 12:00:00 AM,1830,Topanga,BURGLARY,35,M,SINGLE FAMILY DWELLING,NaN,23500 WINDOM ST,15
...,...,...,...,...,...,...,...,...,...,...,...,...
1004891,240710284,07/24/2024 12:00:00 AM,07/23/2024 12:00:00 AM,1400,Wilshire,VEHICLE - STOLEN,0,NaN,STREET,NaN,4000 W 23RD ST,122
1004892,240104953,01/15/2024 12:00:00 AM,01/15/2024 12:00:00 AM,100,Central,VANDALISM - MISDEAMEANOR ($399 OR UNDER),0,X,HOTEL,UNKNOWN WEAPON/OTHER WEAPON,1300 W SUNSET BL,120
1004893,240410786,10/14/2024 12:00:00 AM,10/11/2024 12:00:00 AM,2330,Hollenbeck,"THEFT-GRAND ($950.01 & OVER)EXCPT,GUNS,FOWL,LI...",29,M,RESTAURANT/FAST FOOD,NaN,1700 ALBION ST,111
1004894,240309674,04/24/2024 12:00:00 AM,04/24/2024 12:00:00 AM,1500,Southwest,"ASSAULT WITH DEADLY WEAPON, AGGRAVATED ASSAULT",70,F,SIDEWALK,STICK,FLOWER ST,2
